# Trabalho de Aprendizado Descritivo

In [36]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [37]:
df = pd.read_csv('./processed.csv')

advanced_economies = {
    "Australia", "Belgium", "Canada", "Czech Republic", "France", "Germany",
    "Hong Kong (S.A.R.)", "Ireland", "Israel", "Italy", "Japan", "Netherlands",
    "Portugal", "Singapore", "South Korea", "Spain", "Taiwan",
    "United Kingdom of Great Britain and Northern Ireland",
    "United States of America"
}

emerging_and_developing_economies = {
    "Algeria", "Argentina", "Bangladesh", "Brazil", "Cameroon", "Chile",
    "China", "Colombia", "Ecuador", "Egypt", "Ethiopia", "Ghana", "India",
    "Indonesia", "Iran, Islamic Republic of...", "Kenya", "Malaysia", "Mexico",
    "Morocco", "Nepal", "Nigeria", "Pakistan", "Peru", "Philippines",
    "Poland", "Romania", "Russia", "Saudi Arabia", "South Africa",
    "Sri Lanka", "Thailand", "Tunisia", "Turkey", "Ukraine",
    "United Arab Emirates", "Viet Nam", "Zimbabwe"
}

def set_tier(name):
    if name in advanced_economies:
        return 'Advanced'
    elif name in emerging_and_developing_economies:
        return 'Emerging'
    else: return -1

#Create country status, discard other
df['Country-status'] = df['Q4'].apply(set_tier)
df = df[df['Country-status'] != -1]
df

,Duration (in seconds),Q2,Q3,Q4,Q5,Q6_Coursera,Q6_edX,Q6_Kaggle Learn Courses,Q6_DataCamp,Q6_Fast.ai,...,"Q44_Email newsletters (Data Elixir, O'Reilly Data & AI, etc)","Q44_Reddit (r/machinelearning, etc)","Q44_Kaggle (notebooks, forums, etc)","Q44_Course Forums (forums.fast.ai, Coursera forums, etc)","Q44_YouTube (Kaggle YouTube, Cloud AI Adventures, etc)","Q44_Podcasts (Chai Time Data Science, O’Reilly Data Show, etc)","Q44_Blogs (Towards Data Science, Analytics Vidhya, etc)","Q44_Journal Publications (peer-reviewed journals, conference proceedings, etc)","Q44_Slack Communities (ods.ai, kagglenoobs, etc)",Country-status
0,121,30-34,Man,India,No,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Emerging
1,462,30-34,Man,Algeria,No,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Emerging
2,293,18-21,Man,Egypt,Yes,1,1,0,1,0,...,1,0,1,0,1,1,0,0,0,Emerging
3,851,55-59,Man,France,No,1,0,1,0,0,...,0,0,1,1,0,0,1,0,0,Advanced
4,232,45-49,Man,India,Yes,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Emerging
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,331,22-24,Man,United States of America,Yes,0,0,0,0,0,...,0,0,1,0,1,1,0,1,0,Advanced
23993,330,60-69,Man,United States of America,Yes,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,Advanced
23994,860,25-29,Man,Turkey,No,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,Emerging
23995,597,35-39,Woman,Israel,No,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,Advanced


In [38]:
def filter_df(df, questions):
    cols = [*df]
    selected = []
    for q in questions:
        #Get associated columns
        selected.extend([c for c in cols if c == q or c.startswith(q + '_')])
    return df[selected]

In [39]:
filtered = filter_df(df, ['Country-status', 'Q5', 'Q8', 'Q9', 'Q10','Q11','Q16','Q30'])

df_ = filtered.copy()

print(df_.head())

df_ = pd.get_dummies(df_)

# Gera itens frequentes
print("Gerando itens frequentes...")
frequent_items = apriori(df_, min_support=0.08, use_colnames=True)

print("Gerando regras de associação...")
# Regras de associação
rules = association_rules(frequent_items, metric='lift', min_threshold=0.8)
rules = rules.sort_values(by='lift', ascending=False)

print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_string())

rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_csv('./rules.csv', index=False)

  Country-status   Q5                                                 Q8   Q9  \
0       Emerging   No                                                NaN  NaN   
1       Emerging   No                                    Master’s degree  Yes   
2       Emerging  Yes                                  Bachelor’s degree  NaN   
3       Advanced   No  Some college/university study without earning ...  NaN   
4       Emerging  Yes                                  Bachelor’s degree  NaN   

   Q10_Yes, the research made advances related to some novel machine learning method (theoretical research)  \
0                                                  0                                                          
1                                                  0                                                          
2                                                  0                                                          
3                                                  0                 

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Gerando regras de associação...
                                                                           antecedents                                                                        consequents   support  confidence      lift
0    (Q10_Yes, the research made use of machine learning as a tool (applied research))                                                                           (Q9_Yes)  0.100377    1.000000  4.638591
1                                                                             (Q9_Yes)  (Q10_Yes, the research made use of machine learning as a tool (applied research))  0.100377    0.465610  4.638591
67                                                                (Q8_Doctoral degree)                                                                           (Q9_Yes)  0.087991    0.811630  3.764819
66                                                                            (Q9_Yes)                                                               (Q8_Doctora

## Análise de Linguagens de Programação

In [40]:
languages = filter_df(df, ['Country-status','Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16'])
languages = pd.get_dummies(languages)
languages

print("Gerando itens frequentes...")
frequent_items = fpgrowth(languages, min_support=0.15, use_colnames=True)

frequent_items

Gerando itens frequentes...


/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.730610,(Country-status_Emerging)
1,0.585705,(Q15_Matplotlib )
2,0.304684,(Q16_Under 1 year)
3,0.270455,(Q11_1-3 years)
4,0.161154,(Q13_Notepad++ )
...,...,...
247,0.162531,"(Q13_Visual Studio , Q12_Python)"
248,0.162664,"(Q12_R, Q12_Python)"
249,0.195117,"(Country-status_Emerging, Q11_< 1 years)"
250,0.192364,"(Q12_Python, Q11_< 1 years)"


In [42]:
from mlxtend.frequent_patterns import fpgrowth
from scipy.stats import chi2_contingency, fisher_exact
import pandas as pd

languages = filter_df(df, ['Country-status','Q11', 'Q12', 'Q13', 'Q14', 'Q15'])
languages = languages[languages['Q11'] != "I have never written code"]

target = languages['Country-status']
languages = pd.get_dummies(languages.drop(columns=['Country-status'])) > 0  # força bool

print("Gerando itens frequentes...")
frequent_items = fpgrowth(languages, min_support=0.15, use_colnames=True)

print("Analisando padrões que mais diferem entre 'Emergent' e 'Advanced'...")
results = []

total_emergent = (target == 'Emerging').sum()
total_advanced = (target == 'Advanced').sum()

for _, row in frequent_items.iterrows():
    items = list(row['itemsets'])
    pattern_mask = languages[items].all(axis=1)

    count_emergent = ((pattern_mask) & (target == 'Emerging')).sum()
    count_advanced = ((pattern_mask) & (target == 'Advanced')).sum()
    # total_emergent = (target == 'Emerging').sum()
    # total_advanced = (target == 'Advanced').sum()

    if count_emergent + count_advanced < 5:
        continue  # ignora padrões muito raros

    contingency = [
        [count_emergent, total_emergent - count_emergent],
        [count_advanced, total_advanced - count_advanced]
    ]

    try:
        chi2, p, _, _ = chi2_contingency(contingency)
    except ValueError:
        _, p = fisher_exact(contingency)  # fallback para Fisher

    results.append((row['itemsets'], p, count_emergent / total_emergent, count_advanced / total_advanced))

sorted_results = sorted(results, key=lambda x: x[1])
for itemset, p, freq_emergent, freq_advanced in sorted_results[:10]:
    print(f"Itemset: {itemset}, p={p:.8f}, freq_emergent={freq_emergent:.2%}, freq_advanced={freq_advanced:.2%}")

Gerando itens frequentes...
Analisando padrões que mais diferem entre 'Emergent' e 'Advanced'...
Itemset: frozenset({'Q12_Python', 'Q11_< 1 years'}), p=0.00000000, freq_emergent=25.10%, freq_advanced=10.49%
Itemset: frozenset({'Q11_< 1 years'}), p=0.00000000, freq_emergent=29.60%, freq_advanced=14.08%
Itemset: frozenset({'Q11_1-3 years'}), p=0.00000000, freq_emergent=33.48%, freq_advanced=19.42%
Itemset: frozenset({'Q12_Python', 'Q11_1-3 years'}), p=0.00000000, freq_emergent=30.53%, freq_advanced=17.20%
Itemset: frozenset({'Q12_R'}), p=0.00000000, freq_emergent=17.57%, freq_advanced=28.42%
Itemset: frozenset({'Q12_R', 'Q13_RStudio '}), p=0.00000000, freq_emergent=12.50%, freq_advanced=21.83%
Itemset: frozenset({'Q15_Matplotlib ', 'Q11_1-3 years'}), p=0.00000000, freq_emergent=23.76%, freq_advanced=13.42%
Itemset: frozenset({'Q12_Python', 'Q15_Matplotlib ', 'Q11_1-3 years'}), p=0.00000000, freq_emergent=23.31%, freq_advanced=13.07%
Itemset: frozenset({'Q13_Jupyter Notebook', 'Q11_1-3 ye

In [43]:
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.stats import chi2_contingency, fisher_exact
import pandas as pd
from math import sqrt

# Seleciona e prepara os dados
languages = filter_df(df, ['Country-status','Q11', 'Q12', 'Q13', 'Q14', 'Q15'])
languages = languages[languages['Q11'] != "I have never written code"]
target = languages['Country-status']
languages = pd.get_dummies(languages.drop(columns=['Country-status'])) > 0

# Frequência de padrões e regras
frequent_items = fpgrowth(languages, min_support=0.10, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=0.8)

# Função para intervalo de confiança
z = 1.96  # 95% de confiança
def get_ci(freq, total):
    if total == 0 or freq == 0 or freq == 1:
        return "± 0.00"
    se = sqrt(freq * (1 - freq) / total)
    return f"± {100 * z * se:.2f}"

# Avaliação por grupo
results = []
for _, row in rules.iterrows():
    antecedents = list(row['antecedents'])
    consequents = list(row['consequents'])
    all_items = antecedents + consequents
    mask = languages[all_items].all(axis=1)

    count_emergent = ((mask) & (target == 'Emerging')).sum()
    count_advanced = ((mask) & (target == 'Advanced')).sum()
    total_emergent = (target == 'Emerging').sum()
    total_advanced = (target == 'Advanced').sum()

    if count_emergent + count_advanced < 5:
        continue

    contingency = [
        [count_emergent, total_emergent - count_emergent],
        [count_advanced, total_advanced - count_advanced]
    ]

    try:
        chi2, p, _, _ = chi2_contingency(contingency)
    except ValueError:
        _, p = fisher_exact(contingency)

    support_emergent = count_emergent / total_emergent
    support_advanced = count_advanced / total_advanced

    results.append({
        'antecedent': antecedents,
        'consequent': consequents,
        'support_emergent': support_emergent,
        'support_advanced': support_advanced,
        'diff': abs(support_emergent - support_advanced),
        'p': p,
        'ci_emergent': get_ci(support_emergent, total_emergent),
        'ci_advanced': get_ci(support_advanced, total_advanced)
    })

df_rules = pd.DataFrame(results)
df_rules = df_rules.sort_values(by='diff', ascending=False)

df_results = pd.DataFrame([
    {
        'Antecedent': ', '.join(sorted(r['antecedent'])),
        'Consequent': ', '.join(sorted(r['consequent'])),
        'Support_Emergent (%)': f"{r['support_emergent']*100:.2f} {r['ci_emergent']}",
        'Support_Advanced (%)': f"{r['support_advanced']*100:.2f} {r['ci_advanced']}",
        'Difference (%)': f"{r['diff']*100:.2f}",
        'p-value': f"{r['p']:.2e}"
    }
    for r in df_rules.head(10).to_dict(orient='records')
])

df_results

,Antecedent,Consequent,Support_Emergent (%),Support_Advanced (%),Difference (%),p-value
0,Q11_< 1 years,Q12_Python,25.10 ± 0.70,10.49 ± 0.79,14.62,4.33e-118
1,Q11_1-3 years,Q12_Python,30.53 ± 0.74,17.20 ± 0.97,13.33,1.02e-83
2,"Q11_< 1 years, Q13_Jupyter Notebook",Q12_Python,17.20 ± 0.61,6.62 ± 0.64,10.58,1.50e-84
3,"Q11_1-3 years, Q15_Matplotlib",Q12_Python,23.31 ± 0.68,13.07 ± 0.87,10.23,5.34e-60
4,"Q11_< 1 years, Q15_Matplotlib",Q12_Python,15.93 ± 0.59,5.77 ± 0.60,10.16,9.23e-84
5,"Q11_1-3 years, Q13_Jupyter Notebook",Q12_Python,22.54 ± 0.67,12.60 ± 0.86,9.93,4.72e-58
6,Q13_RStudio,Q12_R,12.50 ± 0.53,21.83 ± 1.07,9.33,4.16e-63
7,"Q11_1-3 years, Q15_Matplotlib",Q13_Jupyter Notebook,19.43 ± 0.64,10.54 ± 0.79,8.89,1.68e-52
8,"Q11_1-3 years, Q13_Jupyter Notebook",Q15_Matplotlib,19.43 ± 0.64,10.54 ± 0.79,8.89,1.68e-52
9,"Q11_1-3 years, Q12_Python, Q15_Matplotlib",Q13_Jupyter Notebook,19.21 ± 0.63,10.40 ± 0.79,8.80,5.05e-52


## Regras de Associação

In [44]:
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.stats import chi2_contingency, fisher_exact
import pandas as pd
from math import sqrt

# Filtra e remove quem nunca programou
languages =  filter_df(df, ['Country-status','Q30', 'Q31', 'Q32', 'Q33', 'Q34','Q35', 'Q36', 'Q37', 'Q38', 'Q39','Q40', 'Q41', 'Q42', 'Q43'])
languages = languages[df['Q11'] != "I have never written code"]
languages = languages.dropna(subset=['Q30'])
languages = languages[languages['Q30'] != "$0 ($USD)"]

# Separa target e aplica one-hot encoding
target = languages['Country-status']
languages = pd.get_dummies(languages.drop(columns=['Country-status'])) > 0

languages = languages.drop(columns=[col for col in languages.columns if "No" in col])
frequent_items = fpgrowth(languages, min_support=0.07, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=0.8)

languages

,Q31_Amazon Web Services (AWS),Q31_Microsoft Azure,Q31_Google Cloud Platform (GCP),Q31_IBM Cloud / Red Hat,Q31_Oracle Cloud,Q31_SAP Cloud,Q31_VMware Cloud,Q31_Alibaba Cloud,Q31_Tencent Cloud,Q31_Huawei Cloud,...,Q32_ SAP Cloud,Q32_ Tencent Cloud,Q32_ VMware Cloud,Q32_Other,Q32_They all had a similarly enjoyable developer experience,Q43_2-5 times,Q43_6-25 times,Q43_More than 25 times,Q43_Never,Q43_Once
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
8,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
13,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False
16,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
17,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23982,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
23983,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
23989,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
23990,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [45]:
from math import sqrt

def get_ci(freq, total):
    if total == 0:
        return "± N/A"
    std_err = sqrt(freq * (1 - freq) / total)
    return f"± {1.96 * 100 * std_err:.2f}"

# Avaliação por grupo + estatísticas
results = []
for _, row in rules.iterrows():
    antecedents = list(row['antecedents'])
    consequents = list(row['consequents'])
    all_items = antecedents + consequents
    mask = languages[all_items].all(axis=1)

    count_emergent = ((mask) & (target == 'Emerging')).sum()
    count_advanced = ((mask) & (target == 'Advanced')).sum()
    total_emergent = (target == 'Emerging').sum()
    total_advanced = (target == 'Advanced').sum()

    if count_emergent + count_advanced < 5:
        continue

    contingency = [
        [count_emergent, total_emergent - count_emergent],
        [count_advanced, total_advanced - count_advanced]
    ]

    try:
        chi2, p, _, _ = chi2_contingency(contingency)
    except ValueError:
        _, p = fisher_exact(contingency)

    support_emergent = count_emergent / total_emergent
    support_advanced = count_advanced / total_advanced

    results.append({
        'antecedent': antecedents,
        'consequent': consequents,
        'support_emergent': support_emergent,
        'support_advanced': support_advanced,
        'diff': abs(support_emergent - support_advanced),
        'p': p,
        'ci_emergent': get_ci(support_emergent, total_emergent),
        'ci_advanced': get_ci(support_advanced, total_advanced),
        'lift': row['lift'],
        'support': row['support'],
        'confidence': row['confidence']
    })

# Converte em DataFrame
df_rules = pd.DataFrame(results)
df_rules = df_rules.sort_values(by='diff', ascending=False)

# Formata os resultados
df_results = pd.DataFrame([
    {
        'Antecedent': ', '.join(sorted(r['antecedent'])),
        'Consequent': ', '.join(sorted(r['consequent'])),
        'Support_Emergent (%)': f"{r['support_emergent']*100:.2f} {r['ci_emergent']}",
        'Support_Advanced (%)': f"{r['support_advanced']*100:.2f} {r['ci_advanced']}",
        'Difference (%)': f"{r['diff']*100:.2f}",
        'p-value': f"{r['p']:.2e}",
        'Lift': f"{r['lift']:.2f}",
        'Confidence': f"{r['confidence']:.2f}",
        'Total Support (%)': f"{r['support']*100:.2f}"
    }
    for r in df_rules.head(50).to_dict(orient='records')
])

# Exibe
df_results.to_csv('./results.csv', index=False)
df_results

,Antecedent,Consequent,Support_Emergent (%),Support_Advanced (%),Difference (%),p-value,Lift,Confidence,Total Support (%)
0,Q34_Amazon Simple Storage Service (S3),Q31_Amazon Web Services (AWS),23.68 ± 1.58,37.08 ± 2.03,13.40,1.50e-24,2.12,0.95,29.55
1,"Q33_Amazon Elastic Compute Cloud (EC2) , Q34_A...",Q31_Amazon Web Services (AWS),19.81 ± 1.48,30.77 ± 1.94,10.96,8.12e-19,2.17,0.97,24.61
2,"Q31_Amazon Web Services (AWS) , Q33_Amazon Ela...",Q34_Amazon Simple Storage Service (S3),19.81 ± 1.48,30.77 ± 1.94,10.96,8.12e-19,2.70,0.84,24.61
3,"Q31_Amazon Web Services (AWS) , Q34_Amazon Sim...",Q33_Amazon Elastic Compute Cloud (EC2),19.81 ± 1.48,30.77 ± 1.94,10.96,8.12e-19,2.70,0.83,24.61
4,Q33_Amazon Elastic Compute Cloud (EC2),Q34_Amazon Simple Storage Service (S3),20.60 ± 1.50,31.23 ± 1.95,10.63,1.63e-17,2.63,0.82,25.25
5,Q34_Amazon Simple Storage Service (S3),Q33_Amazon Elastic Compute Cloud (EC2),20.60 ± 1.50,31.23 ± 1.95,10.63,1.63e-17,2.63,0.81,25.25
6,Q33_Amazon Elastic Compute Cloud (EC2),Q31_Amazon Web Services (AWS),24.72 ± 1.60,35.19 ± 2.01,10.47,1.21e-15,2.11,0.95,29.31
7,"Q34_Amazon Simple Storage Service (S3) , Q42_...",Q31_Amazon Web Services (AWS),15.93 ± 1.36,25.24 ± 1.83,9.31,5.75e-16,2.12,0.95,20.01
8,"Q33_Amazon Elastic Compute Cloud (EC2) , Q42_G...",Q31_Amazon Web Services (AWS),15.72 ± 1.35,24.55 ± 1.81,8.84,9.85e-15,2.10,0.95,19.58
9,"Q31_Amazon Web Services (AWS) , Q33_Amazon Ela...",Q34_Amazon Simple Storage Service (S3),13.28 ± 1.26,21.93 ± 1.74,8.65,1.31e-15,2.81,0.87,17.06


In [46]:
df_results.sort_values(by='Lift', ascending=False)

,Antecedent,Consequent,Support_Emergent (%),Support_Advanced (%),Difference (%),p-value,Lift,Confidence,Total Support (%)
48,"Q31_Amazon Web Services (AWS) , Q35_Amazon RDS",Q34_Amazon Simple Storage Service (S3),6.89 ± 0.94,11.05 ± 1.32,4.17,3.24e-07,3.03,0.94,8.71
49,Q35_Amazon RDS,"Q31_Amazon Web Services (AWS) , Q34_Amazon Sim...",6.89 ± 0.94,11.05 ± 1.32,4.17,3.24e-07,3.01,0.89,8.71
41,"Q33_Amazon Elastic Compute Cloud (EC2) , Q35_P...","Q31_Amazon Web Services (AWS) , Q34_Amazon Sim...",5.78 ± 0.87,10.59 ± 1.29,4.82,5.98e-10,2.83,0.84,7.89
40,"Q31_Amazon Web Services (AWS) , Q33_Amazon Ela...",Q34_Amazon Simple Storage Service (S3),5.78 ± 0.87,10.59 ± 1.29,4.82,5.98e-10,2.82,0.88,7.89
9,"Q31_Amazon Web Services (AWS) , Q33_Amazon Ela...",Q34_Amazon Simple Storage Service (S3),13.28 ± 1.26,21.93 ± 1.74,8.65,1.31e-15,2.81,0.87,17.06
39,"Q34_Amazon Simple Storage Service (S3) , Q35_...","Q31_Amazon Web Services (AWS) , Q33_Amazon Ela...",5.78 ± 0.87,10.59 ± 1.29,4.82,5.98e-10,2.81,0.82,7.89
38,"Q31_Amazon Web Services (AWS) , Q34_Amazon Sim...",Q33_Amazon Elastic Compute Cloud (EC2),5.78 ± 0.87,10.59 ± 1.29,4.82,5.98e-10,2.80,0.87,7.89
46,"Q33_Amazon Elastic Compute Cloud (EC2) , Q35_P...",Q34_Amazon Simple Storage Service (S3),6.17 ± 0.89,10.73 ± 1.30,4.56,8.16e-09,2.79,0.87,8.17
10,"Q33_Amazon Elastic Compute Cloud (EC2) , Q42_G...","Q31_Amazon Web Services (AWS) , Q34_Amazon Sim...",13.28 ± 1.26,21.93 ± 1.74,8.65,1.31e-15,2.79,0.82,17.06
30,"Q34_Amazon Simple Storage Service (S3) , Q35_...","Q31_Amazon Web Services (AWS) , Q33_Amazon Ela...",8.14 ± 1.02,14.19 ± 1.47,6.04,1.41e-11,2.78,0.82,10.79
